In [2]:
import os
from dotenv import load_dotenv
from langchain_community.llms import Ollama
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import OllamaEmbeddings
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import DocArrayInMemorySearch
from operator import itemgetter



class tourism():
    def __init__(self, openai_key = "OPENAI_API_KEY", MODEL="gpt-4o"):
        load_dotenv()
        self.openai_key = openai_key
        self.__OPENAI_API_KEY = os.getenv(self.openai_key)
        self.MODEL = MODEL

        self.parser = StrOutputParser()


        if self.MODEL.startswith("gpt"):
            self.model = ChatOpenAI(api_key=self.__OPENAI_API_KEY, model=self.MODEL)
            self.embeddings = OpenAIEmbeddings()
        else:
            self.model = Ollama(model=self.MODEL)
            self.embeddings = OllamaEmbeddings(model=self.MODEL)

        
        self.template = """
        أنت نموذج من نظام لبيب. مهمتك هي الإجابة على الأسئلة المتعلقة بالسياحة في المملكة العربية السعودية بناءً على المعلومات المتوفرة في الوثائق المحددة.

        احرص على تنوع الترفيه والاماكن الترفيهية،
        إذا كنت لا تعرف الإجابة، أجب بـ "لا أعرف".

        عند الإجابة على السؤال، يرجى الإشارة إلى الأماكن الترفيهية أو المطاعم المتعلقة بالسؤال بالتفصيل. تأكد من الكتابة باللغة العربية الفصحى وتجنب تكرار المطاعم او الاماكن الترفيهية وتجنب الأخطاء الإملائية.

        المعلومات المتاحة: {documents}

        السؤال: {question}
        """


        self.prompt = PromptTemplate.from_template(self.template)



    def inference(self, questions):

        
        pdf_files = [
            "مطاعم-الرياض.pdf",
            "ترفيه-الرياض.pdf",
            "فنادق-الرياض.pdf"
           
        ]

        pages = []
        for pdf_file in pdf_files:
            loader = PyPDFLoader(pdf_file)
            pages.extend(loader.load_and_split())

        vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=self.embeddings)
        retriever = vectorstore.as_retriever()

        chain = (
            {
                "documents": itemgetter("question") | retriever,
                "question": itemgetter("question"),
            }
            | self.prompt
            | self.model
            | self.parser
        )

        
        questions_responses = {}
        for question in questions:
            questions_responses[question] = chain.invoke({'question': question})
        
        return questions_responses
    




if __name__ == '__main__':
    tourism = tourism()
    questions = [
                "انا زائر الرياض لمدة ٣ ايام اعطني خطة سياحية ",
            ]

    q_responses = tourism.inference(questions)
    for question, answer in q_responses.items():
        print("السؤال: ", question)
        print(answer)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 15 0 (offset 0)


Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 9 0 (offset 0)
Ignoring wrong pointing object 13 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)


السؤال:  انا زائر الرياض لمدة ٣ ايام اعطني خطة سياحية 
الإجابة:
يوم ١:
- قم بزيارة متحف القصر المسمك Al Masmak Palace Museum لاكتشاف التاريخ السعودي
- استمتع بالتسوق والترفيه في Blvd World
- تناول العشاء في أحد المطاعم الفاخرة في الفيصلية برج

يوم ٢:
- قم بزيارة حديقة البجيري التراثية Al-Bujairi Heritage Park لقضاء وقت ممتع مع العائلة
- استمتع بألعاب الأطفال والترفيه في Sparky's
- تناول وجبة خفيفة في Magic Planet Riyadh Front

يوم ٣:
- استمتع بالطبيعة في حديقة المرج Al Muruj Park
- قم بزيارة جسر السماء في مركز المملكة للتسوق للاستمتاع بإطلالة رائعة
- تناول وجبة شهية في مطعم Tokyo Restaurant

هذه خطة سياحية تتضمن أماكن ترفيهية متنوعة ومطاعم شهية لتجربة تجارب ممتعة خلال زيارتك للرياض لمدة ٣ أيام.
